In [1]:
# import modules
import os
import re

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import pandas as pd

In [2]:
saved_file = "/home/mingi/data/chartgemma/one_by_one_results/vertical.jsonl"

In [3]:
files = [json.loads(line) for line in open(saved_file, "r")]

In [16]:
# 150
image = Image.open(
    os.path.join(
        "/home/mingi/data/vistext/data/test_images/vertical", files[150]["image_file"]
    )
).convert("RGB")
plt.imshow(image)
plt.plot()
files[150]["image_file"]
files[150]["text"]
files[150]["value_list"]

[1.86, 2.03, 2.68, 0.28, -1.38, 1.84, 2.61, 1.77, 2.74, 2.45]

In [15]:
files[150]["text"]
# files[150]["image_file"]

['Earnings per share in Canadian dollars \n(in Canadian dollars)',
 'The value of 2018 is 1.8.',
 '2.5',
 'The value of 2016 is 2.6 (in Canadian dollars).',
 'Earnings per share in Canadian dollars 2015 = -0.1',
 'The value of 2014 is -0.5.',
 'The value of 2013 is 2.2.',
 'The value of 2012 is 2.5',
 '2011 earns per share is 1.75 Canadian dollars.',
 '2.5']

In [6]:
def extract_last_number(strings):
    result = []
    for string in strings:
        if (
            "not" in string.lower()
            or "no" in string.lower()
            or "Please" in string.lower()
            or "What" in string.lower()
        ):
            result.append(0)
        else:
            # Find all numbers (including negative numbers) in the string
            numbers = re.findall(r"-?\d{1,3}(?:,\d{3})*(?:\.\d+)?", string)

            if numbers:
                # Convert the last found number to an appropriate type (int or float) and add to the result
                last_number_str = numbers[-1].replace(",", "")
                last_number = (
                    float(last_number_str)
                    if "." in last_number_str
                    else int(last_number_str)
                )
                result.append(last_number)
            else:
                result.append(0)  # If no numbers found, return 0
    return result

In [7]:
extract_last_number(files[150]["text"])

[0, 1.8, 2.5, 2.6, -0.1, -0.5, 2.2, 2.5, 1.75, 2.5]

In [8]:
def eval_metric(gt_values, values):

    l = len(gt_values)
    sum_error_rates = 0
    for gt_value, value in zip(gt_values, values):
        if isinstance(gt_value, str):
            try:
                gt_value = float(gt_value)
            except ValueError:
                gt_value = 0

        error = abs(gt_value - value)

        if gt_value == 0:
            l = l - 1
            continue

        error_rate = error / (abs(gt_value))
        error_rate = min(error_rate, 1)
        sum_error_rates = sum_error_rates + error_rate

    avg_error_rates = sum_error_rates / l

    return avg_error_rates

In [9]:
# Evaluation
sum = 0
for file in files:
    texts = file["text"]

    gt_values = file["value_list"]
    values = extract_last_number(texts)

    metric = eval_metric(gt_values, values)

    sum = sum + metric

avg = sum / len(files)

In [10]:
avg

0.35459422027218973